# A Soft Drink Distribution Problem

### Problem Description
A leading soft drink manufacturer has two warehouses from which it distributes soft drink to five carefully chosen distributors. At the start of every week, each distributor sends an order to the company’s head office for so many crates of soft drink, which is then dispatched from the appropriate warehouse to the distributor. The company would like to have an interactive computer program which they can run week by week to tell them which warehouse should supply which distributor so as to minimize the costs of the whole operation. 

**For example, suppose that at the start of a given week the company has 1000 cases at warehouse A, and 4000 cases at warehouse B, and that the distributors require 500, 900, 1800, 200, and 700 cases respectively. Which warehouse should supply which distributor?**

### Formulation
Here is a graphical representation of The Soft Drink Distribution Problem.

<img src="warehouse_distributor.jpg" title="Warehouse - Distributor Setup" />

#### Identifying the Decision Variables
In this transportation problem we are deciding how to transport goods from their supply nodes to their demand nodes. The decision variables are the Arcs connecting these nodes, as shown in the diagram below. We are deciding how many crates of soft drink to transport from each warehouse to each distributor.

<img src="warehouse_distributor_link.jpg" title="Warehouse - Distributor Setup and Link" />

- x1 = number of crates of soft drink to ship from Warehouse A to Distributor 1
- x5 = number of crates of soft drink to ship from Warehouse A to Distributor 5
- y1 = number of crates of soft drink to ship from Warehouse B to Distributor 1
- y5 = number of crates of soft drink to ship from Warehouse B to Distributor 5
 
Let,
Warehouse = {x, y}
Distributor = {1,2,3,4,5}
 
The lower bound on the variables is Zero, and the values must all be Integers (since the number of crates cannot be negative or fractional). There is no upper bound.

#### Formulating the Objective Function
The objective function can be defined as the cost. The problem can only be formulated as a linear program if the cost of transportation from warehouse to distributor is a linear function of the amounts of crates transported. Noting that this is sometimes not the case. This may be due to factors such as economies of scale or fixed costs. For example, transporting 10 crates may not cost 10 times as much as transporting one crate, since it may be the case that one truck can accommodate 10 crates as easily as one. Usually in this situation there are fixed costs in operating a truck which implies that the costs go up in jumps (when an extra truck is required).

We shall assume then that there is a fixed transportation cost per crate. (If the capacity of a truck is small compared with the number of crates that must be delivered then this is a valid assumption). Let's assume the following transportation costs (dollars per crate):

<img src="transportation_cost.jpg" title="Transportation Cost" />

**Minimise the Transporting Costs = Cost per crate for Route x1 * x1 (number of crates on Route x1) + … + Cost per crate for Route y5 * y5 (number of crates on Route y5)**

#### Formulating the Constraints
The constraints come from considerations of supply and demand. The supply of soft drink at warehouse A is 1000 cases. The total amount of soft drink shipped from warehouse A cannot exceed this amount. Similarly, the amount of soft drink shipped from warehouse B cannot exceed the supply of soft drink at warehouse B. The sum of the values on all the arcs leading out of a warehouse, must be less than or equal to the supply value at that warehouse:

Such that:
- x1 + x2 + x3 + x4 + x5 <= 1000
- y1 + y2 + y3 + y4 + y5 <= 4000

The demand for soft drink at distributor 1 is 500 cases, so the amount of soft drink delivered there must be at least 500 to avoid lost sales. Similarly, considering the amounts delivered to the other distributors must be at least equal to the demand at those distributors. Note, we are assuming there are no penalties for oversupplying distributors (other than the extra transportation cost we incur). We can _balance_ the transportation problem to make sure that demand is met exactly. For now, the sum of the values on all the arcs leading into a distributor, must be greater than or equal to the demand value at that distributor:

- x1 + y1 >= 500
- x2 + y2 >= 900
- x3 + y3 >= 1800
- x4 + y4 >= 200
- x5 + y5 >= 700

Finally, we have already specified the amount of soft drink shipped must be non-negative.

In [2]:
# Import PuLP modeler functions
from pulp import *

The start of the formulation is a simple definition of the nodes and their limits/capacities. The node names are put into lists, and their associated capacities are put into dictionaries with the node names as the reference keys:

In [3]:
# Creates a list of all the supply nodes
Warehouses = ["x", "y"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"x": 1000,
          "y": 4000}

# Creates a list of all demand nodes
Distributors = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1":500,
          "2":900,
          "3":1800,
          "4":200,
          "5":700,}

The cost data is then inputted into a list, with two sub lists: the first containing the costs of shipping from Warehouse A, and the second containing the costs of shipping from Warehouse B. The Warehouses and Distributors lists (Supply and Demand nodes) are added to make a large list (of all nodes) and inputted into PuLPs makeDict function. The second parameter is the costs list as was previously created, and the last parameter sets the default value for an arc cost. Once the cost dictionary is created, if costs [A][1] is called, it will return the cost of transporting from warehouse A to distributor 1, 2. If costs [C][2] is called, it will return 0, since this is the defined default.

In [4]:
# Creates a list of costs of each transportation path
costs = [[2,4,5,2,1], #A
         [3,1,3,2,3]  #B
        ]

# The cost data is made into a dictionary
costs = makeDict([Warehouses, Distributors], costs, 0)
costs

defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
            {'x': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 2, '2': 4, '3': 5, '4': 2, '5': 1}),
             'y': defaultdict(<function pulp.utilities.__makeDict.<locals>.<lambda>()>,
                         {'1': 3, '2': 1, '3': 3, '4': 2, '5': 3})})

In [5]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Soft_Drink_Distribution_Problem", LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,d) for w in Warehouses for d in Distributors]
Routes

[('x', '1'),
 ('x', '2'),
 ('x', '3'),
 ('x', '4'),
 ('x', '5'),
 ('y', '1'),
 ('y', '2'),
 ('y', '3'),
 ('y', '4'),
 ('y', '5')]

In [6]:
(Warehouses, Distributors)

(['x', 'y'], ['1', '2', '3', '4', '5'])

In [5]:
# A dictionary called 'Vars' is created to contain the referenced variables(the routes)
vars = LpVariable.dicts("Route", (Warehouses, Distributors), lowBound=0, upBound=None, cat=LpInteger)
vars

{'x': {'1': Route_x_1,
  '2': Route_x_2,
  '3': Route_x_3,
  '4': Route_x_4,
  '5': Route_x_5},
 'y': {'1': Route_y_1,
  '2': Route_y_2,
  '3': Route_y_3,
  '4': Route_y_4,
  '5': Route_y_5}}

In [6]:
# The objective function is added to 'prob' first
prob += lpSum([vars[w][d]*costs[w][d] for (w,d) in Routes]), "Sum_of_Transporting_Costs"

# this basically means, (route x1 * cost from A->1) + (route x2 * cost from A->2) + .. + (route x5 * cost from B->5)

In [7]:
# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += lpSum([vars[w][d] for d in Distributors])<=supply[w], "Sum_of_Products_out_of_Warehouse_%s"%w
# this basically means, (route x1) + (route x2) + .. + (route x5) <= 1000 and,
#                       (route y1) + (route y2) + .. + (route y5) <= 4000

# The demand minimum constraints are added to prob for each demand node (distributor)
for d in Distributors:
    prob += lpSum([vars[w][d] for w in Warehouses])>=demand[d], "Sum_of_Products_into_Distributor_%s"%d
# this basically means, (route x1) + (route y1) >= 500,
#                       (route x2) + (route y2) >= 900,
#                       (route x3) + (route y3) >= 1800,
#                       (route x4) + (route y4) >= 200 and,
#                       (route x5) + (route y5) >= 700

In [10]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen    
print("Total Cost of Transportation = ", value(prob.objective))

Status: Optimal
Route_x_1 = 300.0
Route_x_2 = 0.0
Route_x_3 = 0.0
Route_x_4 = 0.0
Route_x_5 = 700.0
Route_y_1 = 200.0
Route_y_2 = 900.0
Route_y_3 = 1800.0
Route_y_4 = 200.0
Route_y_5 = 0.0
Total Cost of Transportation =  8600.0
